In [22]:
import pandas as pd
from sklearn import * 
import numpy as np

In [23]:
df = pd.read_csv("/data/kaggle/house-prices/data_combined_cleaned.csv")
del df["Id"]
df = df[~df.SalesPrice.isna()]

In [24]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [31]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()

del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])

pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
print(
    "training MSE", metrics.mean_squared_error(y_train, y_train_pred),
    "\ntesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
    "\ntraining R2", metrics.r2_score(y_train, y_train_pred),
    "\ntesting R2", metrics.r2_score(y_test, y_test_pred)
     )

training MSE 0.007839665256359247 
testing MSE 2.1428968502212463e+24 
training R2 0.946504439067347 
testing R2 -1.1338708971948151e+25


In [36]:
est1 = pipe.steps[-1][-1]

In [50]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()

del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])

pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
print(
    "training MSE", metrics.mean_squared_error(y_train, y_train_pred),
    "\ntesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
    "\ntraining R2", metrics.r2_score(y_train, y_train_pred),
    "\ntesting R2", metrics.r2_score(y_test, y_test_pred)
     )

training MSE 0.015527039450020465 
testing MSE 0.022954852934347732 
training R2 0.8940480673803641 
testing R2 0.8785389987905549


In [51]:
est2 = pipe.steps[-1][-1]

In [52]:
res = pd.DataFrame({"feature": X.columns, "weight1": est1.coef_, "weight2": est2.coef_})

In [53]:
res

,feature,weight1,weight2
0,MSSubClass,-3.701882e-03,-0.014961
1,LotFrontage,5.397192e-03,-0.000000
2,LotArea,2.783767e-02,0.011859
3,OverallQual,6.426097e-02,0.104135
4,OverallCond,3.652346e-02,0.023309
5,YearBuilt,5.423834e-02,0.029465
6,YearRemodAdd,1.028680e-02,0.021713
7,MasVnrArea,-5.711470e-04,0.000000
8,BsmtFinSF1,-4.516660e+12,0.000000
9,BsmtFinSF2,-1.607361e+12,0.000000


In [43]:
res.query("weight1>0").shape

(138, 3)

In [44]:
res.query("weight2>0").shape

(32, 3)

In [46]:
res.query("weight2>0").sort_values("weight2", ascending = False)

,feature,weight1,weight2
3,OverallQual,6.426097e-02,0.104135
15,GrLivArea,2.081861e+12,0.095563
25,GarageCars,1.346122e-03,0.055896
5,YearBuilt,5.423834e-02,0.029465
4,OverallCond,3.652346e-02,0.023309
6,YearRemodAdd,1.028680e-02,0.021713
70,Neighborhood_NridgHt,7.846846e-03,0.020148
60,Neighborhood_Crawfor,1.307914e-02,0.019489
16,BsmtFullBath,9.957632e-03,0.018030
157,Foundation_PConc,2.226753e-02,0.012625


In [54]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()

del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=100)) 
# Lasso generally requires lower value of alpha compared to ridge
])

pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
print(
    "training MSE", metrics.mean_squared_error(y_train, y_train_pred),
    "\ntesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
    "\ntraining R2", metrics.r2_score(y_train, y_train_pred),
    "\ntesting R2", metrics.r2_score(y_test, y_test_pred)
     )

training MSE 0.009761675134804183 
testing MSE 0.023037894954072175 
training R2 0.9333892111585905 
testing R2 0.8780995985954401


In [55]:
res["weight3"] = pipe.steps[-1][-1].coef_

In [57]:
res.sort_values("weight3", ascending = False)

,feature,weight1,weight2,weight3
3,OverallQual,6.426097e-02,0.104135,0.056028
15,GrLivArea,2.081861e+12,0.095563,0.044240
12,1stFlrSF,-1.542132e+12,0.007722,0.031367
70,Neighborhood_NridgHt,7.846846e-03,0.020148,0.029132
4,OverallCond,3.652346e-02,0.023309,0.028937
110,RoofMatl_CompShg,3.345845e-01,0.000000,0.028912
25,GarageCars,1.346122e-03,0.055896,0.028208
13,2ndFlrSF,-1.710080e+12,0.000000,0.024812
60,Neighborhood_Crawfor,1.307914e-02,0.019489,0.023958
116,RoofMatl_WdShngl,1.516925e-01,0.001826,0.022951


In [59]:
target = "SalesPrice"
y = np.log(df[target])
X = df.copy()

del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha=0.01, l1_ratio=0.8)) 
# Lasso generally requires lower value of alpha compared to ridge
])

pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
print(
    "training MSE", metrics.mean_squared_error(y_train, y_train_pred),
    "\ntesting MSE", metrics.mean_squared_error(y_test, y_test_pred),
    "\ntraining R2", metrics.r2_score(y_train, y_train_pred),
    "\ntesting R2", metrics.r2_score(y_test, y_test_pred)
     )

training MSE 0.014493763338268465 
testing MSE 0.022592026250985338 
training R2 0.9010988384769548 
testing R2 0.8804588234286277


In [61]:
_,p_vals = feature_selection.f_regression(X_train, y_train)

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [63]:
res["pval"] = p_vals

Lower pvalue indicates higher significance of the the feature in the model.
p-value for significant feature must be < 0.01

In [65]:
res.sort_values("pval")

,feature,weight1,weight2,weight3,pval
3,OverallQual,6.426097e-02,0.104135,0.056028,1.543610e-232
15,GrLivArea,2.081861e+12,0.095563,0.044240,1.502663e-147
25,GarageCars,1.346122e-03,0.055896,0.028208,8.980028e-140
26,GarageArea,3.691761e-02,0.000000,0.017144,9.673012e-121
18,FullBath,6.548328e-03,0.011998,0.020440,7.435435e-106
11,TotalBsmtSF,4.369642e+12,0.004723,0.006796,4.707153e-98
151,ExterQual_TA,-2.189242e-02,-0.005217,-0.010922,1.996594e-96
12,1stFlrSF,-1.542132e+12,0.007722,0.031367,1.957045e-95
5,YearBuilt,5.423834e-02,0.029465,0.018237,2.946855e-85
6,YearRemodAdd,1.028680e-02,0.021713,0.016315,3.840149e-80


# Find non significant features

In [66]:
res.query("pval > 0.01")

,feature,weight1,weight2,weight3,pval
4,OverallCond,3.652346e-02,0.023309,0.028937,0.034816
9,BsmtFinSF2,-1.607361e+12,0.000000,0.006376,0.924248
14,LowQualFinSF,-1.888455e+11,-0.000000,0.006882,0.662835
17,BsmtHalfBath,1.081010e-03,0.000000,-0.000614,0.680571
30,3SsnPorch,4.210103e-03,0.000000,0.004271,0.309722
32,PoolArea,3.365972e-02,-0.000000,0.020648,0.245086
33,MiscVal,2.242606e-02,-0.000000,0.002914,0.409061
34,MoSold,7.924387e-04,0.000000,0.003274,0.038940
35,YrSold,-1.262172e-03,-0.000000,-0.002991,0.844464
37,MSZoning_RH,2.742848e-02,0.000000,0.005404,0.111989


In [74]:
print("Lasso:", list(res.sort_values("weight1", ascending= False).feature[:10]))
print("\nRidge:", list(res.sort_values("weight2", ascending= False).feature[:10]))
print("\nElasticNet:", list(res.sort_values("weight3", ascending= False).feature[:10]))
print("\npval:", list(res.sort_values("pval").feature[:10]))

Lasso: ['TotalBsmtSF', 'GrLivArea', 'Condition2_RRAe', 'BsmtCond_Po', 'BsmtFinType1_None', 'GarageCond_None', 'Exterior2nd_CBlock', 'HeatingQC_Po', 'RoofMatl_Metal', 'Exterior1st_AsphShn']

Ridge: ['OverallQual', 'GrLivArea', 'GarageCars', 'YearBuilt', 'OverallCond', 'YearRemodAdd', 'Neighborhood_NridgHt', 'Neighborhood_Crawfor', 'BsmtFullBath', 'Foundation_PConc']

ElasticNet: ['OverallQual', 'GrLivArea', '1stFlrSF', 'Neighborhood_NridgHt', 'OverallCond', 'RoofMatl_CompShg', 'GarageCars', '2ndFlrSF', 'Neighborhood_Crawfor', 'RoofMatl_WdShngl']

pval: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'FullBath', 'TotalBsmtSF', 'ExterQual_TA', '1stFlrSF', 'YearBuilt', 'YearRemodAdd']


In [79]:
scores = model_selection.cross_val_score(pipe, X=X_train, 
                        y= y_train, cv=5, scoring="neg_mean_squared_error")

In [80]:
scores

array([-0.05776941, -0.01657383, -0.02707862, -0.01407442, -0.01288614])

In [82]:
np.mean(scores)

-0.025676484253987213

In [83]:
pipe

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.8,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))])

In [105]:
param_grid = {
    "est__l1_ratio": np.linspace(0, 1, 10)
}

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha=0.1, max_iter=10000, tol=1e-3, l1_ratio=0.8)) 
])

gsearch = model_selection.GridSearchCV(cv=5, scoring="neg_mean_squared_error",
                        estimator=pipe, n_jobs=1, param_grid=param_grid)
gsearch.fit(X_train, y_train)
est = gsearch.best_estimator_
print("Test MSE:", metrics.mean_squared_error(y_test, est.predict(X_test)), 
      "\nTrain MSE", - gsearch.best_score_,
      "\nparams:", gsearch.best_params_)

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/abulbasar/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Obje

Test MSE: 0.024119398766279238 
Train MSE 0.024710933063358222 
params: {'est__l1_ratio': 0.1111111111111111}


In [85]:
np.linspace(0, 1, 10)

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

In [88]:
gsearch.best_params_

{'est__l1_ratio': 0.8888888888888888}

In [89]:
gsearch.best_score_

0.8242553069644629

In [90]:
est = gsearch.best_estimator_
est

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True,
      l1_ratio=0.8888888888888888, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False))])

0.022736765792567764